In [19]:
import folium
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

In [20]:
checklists = pd.read_csv("../effort_plus_swamp_checklists.csv")
checklists = checklists[checklists["date"].isin(["2017-05-11", "2017-05-12", "2017-05-13"])]

In [21]:
cl = checklists[['sub_id', 'species_codes',"locality_id", "latitude.x", "longitude.x", "datetime", "effort_hrs"]]

In [10]:
#add end time

In [22]:
cl["end_time"] = ""
cl= cl.reset_index()

/tmp/ipykernel_175864/1544355425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cl["end_time"] = ""


In [23]:
for i, row in cl.iterrows():
    endtime = datetime.fromisoformat(row["datetime"]) + timedelta(hours = row["effort_hrs"])
    cl.loc[i, "end_time"] = str(endtime)

In [43]:
cl2 = cl.copy()

In [28]:
cl2["species_code"] = cl2["species_codes"].map(lambda x: x.split(" "))

In [34]:

# Repeat the index according to the length of each list in split_values
repeated_index = cl2.index.repeat(cl2["species_code"].str.len())



In [56]:
df_final = pd.DataFrame(columns=list(cl2.columns))

In [62]:
for i, row in cl2.iterrows():
    species_list = row["species_codes"].split(" ")
    temp = pd.DataFrame({"species_codes": species_list})
    for c in cl2.columns:
        if c != "species_codes":
            temp[c] = row[c]
    df_final = pd.concat([df_final, temp])
        


/tmp/ipykernel_175864/3474879728.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, temp])


# Spatial mapping of ARUs to hotspots

In [ ]:
import geopy

In [67]:
checklists = df_final

In [84]:
uniques = checklists[["locality_id", "latitude.x", "longitude.x"]].drop_duplicates(keep="first")

In [86]:
hotspots_locs = list(zip(uniques["locality_id"],uniques["latitude.x"], uniques["longitude.x"]))

In [82]:
audio =  pd.read_csv("/network/scratch/t/tengmeli/umich_swamp_aru_anaylsis/swamp_aru_recorder_info.csv")
audio.head()

,site_num,swift_num,latitude,longitude
0,1,28,42.48167,-76.4601
1,2,29,42.48185,-76.4582
2,3,27,42.48182,-76.4560
3,4,24,42.48052,-76.4613
4,5,17,42.48056,-76.4599


In [83]:
audio_locs =  list(zip(audio["latitude"], audio["longitude"]))

In [95]:
import pandas as pd
import numpy as np

# Define the Haversine function to calculate the distance between two points on the Earth's surface
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance


# Convert to DataFrame for easier manipulation
df1 = uniques #hotspots
df2 = audio
# Find the closest location in list2 for each point in list1
closest_locations = {}

for i, row1 in df1.iterrows():
    min_distance = float('inf')
    closest_location = None
    for j, row2 in df2.iterrows():
        distance = haversine(row1['latitude.x'], row1['longitude.x'], row2['latitude'], row2['longitude'])
        if distance < min_distance:
            min_distance = distance
            closest_locations[row1.locality_id] = row2.site_num
            print(row1.locality_id, min_distance)

# Output the closest locations
print(closest_locations)


L1498729 0.47009290324820646
L1498729 0.3593022268702824
L1498729 0.264762976020859
L1498729 0.24990342924959721
L1498729 0.1641900615468234
L1498729 0.09243073713477985
L1498729 0.06855140950776036
L1477502 1.0628928877467125
L1477502 0.9133074166493502
L1477502 0.7368754889507177
L1477502 0.6397899704548484
L1477502 0.5509217235490453
L1477502 0.40962352688021686
L1477502 0.36258031262501883
L1477502 0.12295623088822642
L1498735 1.0733531621914212
L1498735 0.9429926362525902
L1498735 0.7884441005486428
L1498735 0.7563474483997861
L1498735 0.6400842259596622
L1498735 0.6019622637167761
L1498735 0.48329350516858094
L1498735 0.48049331352477587
L1498735 0.47503066876375905
L1498735 0.15984128984452786
L1498735 0.13224873645294014
L97555 0.6723525674636536
L97555 0.5531973130708586
L97555 0.42409653309992285
L97555 0.3595798253004212
L97555 0.25183430796867523
L97555 0.24289343153631862
L97555 0.11885093218837002
L97555 0.07313193279479388
L1246441 0.580253284448978
L1246441 0.4376818102

In [98]:
df_final.columns

Index(['index', 'sub_id', 'species_codes', 'locality_id', 'latitude.x',
       'longitude.x', 'datetime', 'effort_hrs', 'end_time', 'closest_ARU_id'],
      dtype='object')

In [112]:
df_final = df_final.reset_index(drop=True)

In [113]:
df_final["closest_ARU_id"] = ""

In [114]:
for i, row in df_final.iterrows():
    df_final.loc[i, "closest_ARU_id"] = closest_locations[row["locality_id"]]

In [117]:
df_final.to_csv("ebird_checklists_species_breakdown_with_start_end_time.csv")

In [118]:
df_final

,index,sub_id,species_codes,locality_id,latitude.x,longitude.x,datetime,effort_hrs,end_time,closest_ARU_id
0,499,S36728358,yetvir,L1498729,42.479520,-76.455164,2017-05-11 06:57:00,1.00,2017-05-11 07:57:00,14.0
1,499,S36728358,bkbwar,L1498729,42.479520,-76.455164,2017-05-11 06:57:00,1.00,2017-05-11 07:57:00,14.0
2,499,S36728358,boboli,L1498729,42.479520,-76.455164,2017-05-11 06:57:00,1.00,2017-05-11 07:57:00,14.0
3,499,S36728358,buhvir,L1498729,42.479520,-76.455164,2017-05-11 06:57:00,1.00,2017-05-11 07:57:00,14.0
4,499,S36728358,wooduc,L1498729,42.479520,-76.455164,2017-05-11 06:57:00,1.00,2017-05-11 07:57:00,14.0
...,...,...,...,...,...,...,...,...,...,...
885,551,S36874312,rewbla,L1246441,42.480406,-76.453235,2017-05-13 05:40:00,0.25,2017-05-13 05:55:00,15.0
886,551,S36874312,grbher3,L1246441,42.480406,-76.453235,2017-05-13 05:40:00,0.25,2017-05-13 05:55:00,15.0
887,551,S36874312,rebwoo,L1246441,42.480406,-76.453235,2017-05-13 05:40:00,0.25,2017-05-13 05:55:00,15.0
888,551,S36874312,norcar,L1246441,42.480406,-76.453235,2017-05-13 05:40:00,0.25,2017-05-13 05:55:00,15.0


# WIP

In [28]:
locations =[]
for i, row in checklists.iterrows():
    lat = row["latitude.x"]
    lon = row["longitude.x"]
    locations.append( {"name": row["sub_id"], "lat": lat, "lon": lon})
m = folium.Map(location=[locations[0]["lat"], locations[0]["lon"]], zoom_start=4)
# Add markers to the map
for location in locations:
    folium.Marker(
        location=[location["lat"], location["lon"]],
        popup=location["name"]
    ).add_to(m)

for location in audio_locations:
    folium.Marker(
        location=[location["lat"], location["lon"]],
        popup=location["name"], icon=folium.Icon(color="red")
    ).add_to(m)

In [29]:
m